In [1]:
import geopandas
import pyproj

proj_str = '+proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'

taxi_zones = geopandas.read_file('../data/taxi_zones.dbf')
streets = geopandas.read_file('../data/geo_export.dbf')

In [2]:
streets.geometry.bounds.describe()

,minx,miny,maxx,maxy
count,119350.000000,119350.000000,119350.000000,119350.000000
mean,-73.920220,40.707423,-73.919458,40.708053
std,0.116279,0.089759,0.116250,0.089739
min,-74.254956,40.497878,-74.254531,40.498045
25%,-73.980374,40.637410,-73.979543,40.638168
50%,-73.910705,40.708103,-73.909910,40.708606
75%,-73.837434,40.764867,-73.836928,40.765521
max,-73.700598,40.914978,-73.700020,40.915103


In [3]:
from functools import partial
from shapely.ops import transform as shapely_transform

proj_src = pyproj.Proj(proj_str, preserve_units=True)
proj_dst = pyproj.Proj(init="epsg:4326")
proj_func = partial(pyproj.transform, proj_src, proj_dst)

taxi_zones['geometry'] = taxi_zones['geometry'].apply(lambda x: shapely_transform(proj_func, x))
taxi_zones['coords'] = taxi_zones['geometry'].apply(lambda x: x.representative_point().coords[0])

In [30]:
import tqdm
import itertools
import numpy as np
import shapely.geometry as geom
import shapely.ops as shapely_ops

segments = []
coord_set = set()
for street in tqdm.tqdm_notebook(streets.geometry):
    coords = list(street.coords)
    for i in range(len(coords) - 1):
        coord_x_0 = np.round(coords[i][0], 5)
        coord_y_0 = np.round(coords[i][1], 5)
        coord_x_1 = np.round(coords[i+1][0], 5)
        coord_y_1 = np.round(coords[i+1][1], 5)
        coord_0 = (coord_x_0, coord_y_0)
        coord_1 = (coord_x_1, coord_y_1)
        coord_set.add(coord_0)
        coord_set.add(coord_1)
        segments.append((coord_0, coord_1))


KeyboardInterrupt: 

In [31]:
coord_zone_map = {}
for coord in tqdm.tqdm_notebook(coord_set):
    coord_zone_map[coord] = taxi_zones.geometry.contains(geom.Point(coord)).nonzero()[0].tolist()

In [33]:
new_segs = []
for seg in tqdm.tqdm_notebook(segments):
    src, dst = seg
    src_zones, dst_zones = coord_zone_map[src], coord_zone_map[dst]
    if len(src_zones) > 0 and len(dst_zones) > 0:
        new_segs.extend((s, d) for s, d in itertools.product(src_zones, dst_zones))
    elif len(src_zones) > 0:
        new_segs.extend((s, dst) for s in src_zones)
    elif len(dst_zones) > 0:
        new_segs.extend((src, d) for d in dst_zones)
    else:
        new_segs.append((src, dst))

In [35]:
len(new_segs)

398570

In [36]:
import networkx as nx

g = nx.Graph()
g.add_edges_from(new_segs)